In [6]:
# get vgg16 model and pop the last layer
from keras.applications.vgg16 import VGG16
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg16.layers.pop()
# save the model
vgg16.save('vgg16.h5')

In [36]:
# deployment of Image captioning model

import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
# import vgg16
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

# load the model
# model = load_model('model_9.h5')

# load image from file
image = image.load_img('image.jpg', target_size=(224, 224))
 
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model

def preprocess_image(image_path):
    
    # Load VGG16 model with desired output layer
    model = VGG16()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    
    # Load and preprocess image
    npix = 224
    target_size = (npix, npix, 3)
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = preprocess_input(image)
    
    # Make a prediction using VGG16 model
    prediction = model.predict(image.reshape((1,) + image.shape[:3]))
    
    return prediction.flatten()

x = preprocess_image('image.jpg')
x = x.reshape(1, -1)
x.shape


1/1 [==============================] - 3s 3s/step


(1, 4096)

In [37]:
loaded_Model = tf.keras.models.load_model('my_model.h5', compile=False)
loaded_Model.compile(loss='categorical_crossentropy', optimizer='adam')

loaded_Model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 30, 64)       286464      ['input_7[0][0]']                
                                                                                                  
 input_6 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 CaptionFeature (LSTM)          (None, 256)          328704      ['embedding[0][0]']              
                                                                                            

In [39]:
import numpy as np
import pickle


def predict_caption(image, tokenizer, model, maxlen):
    '''
    image.shape = (1, 4462)
    '''
    in_text = 'startseq'


    for iword in range(maxlen):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen)
        yhat = model.predict([image, sequence], verbose=0)
        yhat = np.argmax(yhat)
        newword = index_word[yhat]
        in_text += " " + newword
        if newword == "endseq":
            break
    return (in_text)


tokenizer = pickle.load(open('tokenizer.pkl', 'rb'))
index_word = dict([(index, word)
                  for word, index in tokenizer.word_index.items()])

predict_caption(x, tokenizer, loaded_Model, 30)


'startseq brown dog is running in the grass endseq'